In [1]:
#Importamos las bibliotecas que utilizaremos
import pandas as pd
import numpy as np
import matplotlib as pl
import matplotlib.pyplot as plt

In [2]:
#Cargamos los datos que utilizaremos
df0=pd.read_csv('datos.dat')

In [3]:
#Cramos un nuevo DF con los datos que utilizaremos del DF anterior
d = {'FECHA':df0.TIMESTAMP,'AIRTEMP':df0.AirTC_Avg}
df = pd.DataFrame(d)
df.head()

,FECHA,AIRTEMP
0,2018-05-11 20:10:00,23.50
1,2018-05-11 20:20:00,22.96
2,2018-05-11 20:30:00,22.73
3,2018-05-11 20:40:00,22.40
4,2018-05-11 20:50:00,22.46


In [4]:
#Verificamos si la fecha es tipo fecha, si no, la cambiamos a tipo fecha
df.dtypes

FECHA       object
AIRTEMP    float64
dtype: object

In [5]:
#Cambiamos de object a date el tipo de la fecha
df['FECHA']=pd.to_datetime(df.apply(lambda x: x['FECHA'],1), dayfirst=True)
df.head()

,FECHA,AIRTEMP
0,2018-05-11 20:10:00,23.50
1,2018-05-11 20:20:00,22.96
2,2018-05-11 20:30:00,22.73
3,2018-05-11 20:40:00,22.40
4,2018-05-11 20:50:00,22.46


In [6]:
#Filtramos nuestros datos con los datos que usaremos solamente (del 01/11/18 a 28/02/19)
df = df[df.FECHA > pd.datetime(2018,11,1)] 

In [7]:
#Creamos nuevas columnas para los años, meses, dias, horas y minitos
df['MINUTO']= df['FECHA'].dt.minute
df['HORA']  = df['FECHA'].dt.hour
df['DIA']   = df['FECHA'].dt.day
df['MES']   = df['FECHA'].dt.month
df['AÑO']   = df['FECHA'].dt.year

#Quitamos la columna de FECHA, por el momento no la ocuparemos
df2=df.drop(['FECHA'],1)

#Ajustos los indices de nuestro DF
f=len(df2)
df2.index = np.arange(0,f)

df2.head()

,AIRTEMP,MINUTO,HORA,DIA,MES,AÑO
0,8.89,10,0,1,11,2018
1,8.66,20,0,1,11,2018
2,8.52,30,0,1,11,2018
3,8.47,40,0,1,11,2018
4,8.58,50,0,1,11,2018


In [8]:
#Sacamos la temperatura de promedio del aire por cadad hora
df2['AIRTEMP']=df2.groupby(["HORA","DIA","MES","AÑO"])["AIRTEMP"].transform("mean")

#Quitamos las columas de los minutos, ya que ya no las necesitamos
df2=df2.drop(['MINUTO'],1)

df2.head()

,AIRTEMP,HORA,DIA,MES,AÑO
0,8.624,0,1,11,2018
1,8.624,0,1,11,2018
2,8.624,0,1,11,2018
3,8.624,0,1,11,2018
4,8.624,0,1,11,2018


In [9]:
#Eliminamos los datos repetidos de nuestro DF
df2=df2.drop_duplicates()

#Ajustamos los indices de nuestro DFn
n=len(df2)
df2.index=np.arange(0,n)
df2.head()

,AIRTEMP,HORA,DIA,MES,AÑO
0,8.624000,0,1,11,2018
1,8.493333,1,1,11,2018
2,8.690000,2,1,11,2018
3,8.846667,3,1,11,2018
4,7.397500,4,1,11,2018


In [16]:
# Calculamos la columna de UF24
count=[]
for i in range (0,len(df2)):
    rr=df2.AIRTEMP[i]
    if (rr<1.4) :
        j=0
        
    if (rr>=1.4 and rr<=2.4) :
        j=0.5   
        
    if (rr>2.4 and rr<=9.1) :
        j=1
        
    if (rr>9.1 and rr<=12.4) :
        j=0.5
        
    if (rr>12.4 and rr<=15.9) :
        j=0
        
    if (rr>15.9 and rr<=18) :
        j=-0.5
        
    if (rr>18) :
        j=-1

    count.append(j)
df2["UF24"]=count
df2.head()

,AIRTEMP,HORA,DIA,MES,AÑO,UF24
0,8.624000,0,1,11,2018,1.0
1,8.493333,1,1,11,2018,1.0
2,8.690000,2,1,11,2018,1.0
3,8.846667,3,1,11,2018,1.0
4,7.397500,4,1,11,2018,1.0
